In [74]:
################################################
### all the packages we need are defined here ##
################################################
using Logging
using Dates
using CSV
using DataFrames
using JLD2
using FileIO
using DataFrames
using Plots 
using CSV
using Distributions
using Random
using Tables
using DelimitedFiles
using BSON: @save, @load

In [5]:
mutable struct cell
    generation::Int64
    ID::Int64
    motherCellID::Int64
    founderCellID::Int64
    ODEFile::String
    fate::Int64 #0 = unknown, 1 = division, 2 = death
    tBirth::Float64
    tDeath::Float64
    solutionObject #the entire solution object. If you don't need this comment it out for efficiency.
    y0::Matrix{Float64} #initial conditions.
    yCoord::Float64 #for plotting lineage trees
    leftDaughterID::Int64
    rightDaughterID::Int64
    parameterList
    parameterNameList::Vector{String}
    cellsRepresented::Int64
    #newSwitch::Int64
end    

In [2]:
function createEmptyCellDF()
    cell_df=DataFrame()

    @load "SSDoutputs/allMutSims/DLBCL-LS1065/all_mut/processedCells/cell_1.jld2" thisCell
    sixhr_df = thisCell.solutionObject[(thisCell.solutionObject.timestamp .== 361.0), :]
    sixhr_df = sixhr_df[!, 2:195]
    # add patient ID
    insertcols!(sixhr_df, 1, :patient_ID => "DLBCL-LS1065")
    delete!(sixhr_df, [1])
    return(sixhr_df)
end

createEmptyCellDF (generic function with 1 method)

In [27]:
function getFounderCells(files, folder, sixhour_df, condition)
    for file in files
        count=length(files)
        wrong=0
        correctCell="cell_1.jld2"
        try
            @load folder*"/"*file*"/"*condition*"/processedCells/cell_1.jld2" thisCell 
            if thisCell.founderCellID!=1 || thisCell.generation!=1
                println("cell missmatch in patient: "*file)
                println(string(thisCell.founderCellID))
                println(string(thisCell.generation))
                thisPatFiles=readdir(folder*"/"*file*"/"*condition*"/processedCells/")
                for thisPatFile in thisPatFiles
                    @load folder*"/"*file*"/"*condition*"/processedCells/"*thisPatFile thisCell 
                    if thisCell.founderCellID==1 && thisCell.generation==1
                        println("found true cell 1: "*thisPatFile)
                        correctCell=thisPatFile
                        continue
                    end
                end
                wrong=wrong+1
            end
       @load folder*"/"*file*"/"*condition*"/processedCells/"*correctCell thisCell 
        temp_df = thisCell.solutionObject[(thisCell.solutionObject.timestamp .== 361.0), :]
        temp_df = temp_df[!, 2:195]
        # add patient ID
        insertcols!(temp_df, 1, :patient_ID => file)
        append!(sixhour_df, temp_df)
        catch
            println("couldn't process patient: "*file)
        end
    end
    return(sixhour_df)
end

getFounderCells (generic function with 1 method)

In [53]:
C2_strict_multi_6h = createEmptyCellDF()

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [54]:
files = readdir("SSDoutputs/C2_strict_sims/")

248-element Vector{String}:
 "P104"
 "P105"
 "P109"
 "P111"
 "P112"
 "P115"
 "P117"
 "P120"
 "P122"
 "P124"
 "P13"
 "P132"
 "P137"
 ⋮
 "P849"
 "P861"
 "P880"
 "P881"
 "P887"
 "P89"
 "P894"
 "P912"
 "P92"
 "P924"
 "P95"
 "P96"

In [55]:
C2_strict_multi_6h = getFounderCells(files, "SSDoutputs/C2_strict_sims/", C2_strict_multi_6h, "all_mut")

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,P104,6.80557,15.7518,5.55172,88.5879,0.100765,0.0386653,1.46904
2,P105,10.4545,24.1281,10.4518,88.2749,0.144238,0.0441876,1.85993
3,P109,10.4528,24.1272,10.4453,88.2629,0.144226,0.0440745,1.86013
4,P111,6.80587,15.751,5.55325,64.4738,0.100768,0.0386937,1.46889
5,P112,10.4532,24.1287,10.4459,88.2643,0.144227,0.0440746,1.86023
6,P115,10.4532,24.1287,10.4459,88.2643,0.144227,0.0440746,1.86023
7,P117,6.80431,15.7506,5.54724,64.4627,0.100756,0.0385746,1.46915
8,P120,9.81549,22.2493,9.89401,84.8452,0.137409,0.0447539,2.04555
9,P122,8.41931,19.6415,7.62849,74.8121,0.120173,0.0413565,1.63945


In [10]:
C2_allmuts_strict = DataFrame(CSV.File("C2_allmuts_strict.csv"))

,PID,Gene,Alt,Param,Amount
,String7,String15,String31,String31,Float64
1,P50,PRDM1,PRDM1:S76*,k1_s_Blimp1-differentiation,0.5
2,P69,PRDM1,PRDM1:N172fs*17,k1_s_Blimp1-differentiation,0.5
3,P75,PRDM1,PRDM1:R442fs*64,k1_s_Blimp1-differentiation,0.5
4,P122,PRDM1,PRDM1:S504fs*2,k1_s_Blimp1-differentiation,0.5
5,P151,PRDM1,PRDM1:W49*,k1_s_Blimp1-differentiation,0.5
6,P216,PRDM1,PRDM1:S76*,k1_s_Blimp1-differentiation,0.5
7,P252,PRDM1,PRDM1:Y60*,k1_s_Blimp1-differentiation,0.5
8,P252,PRDM1,PRDM1:G438fs*68,k1_s_Blimp1-differentiation,0.5
9,P254,PRDM1,PRDM1:L83fs*9,k1_s_Blimp1-differentiation,0.5


In [57]:
unique(C2_allmuts_strict.PID)

564-element Vector{String7}:
 "P50"
 "P69"
 "P75"
 "P122"
 "P151"
 "P216"
 "P252"
 "P254"
 "P273"
 "P289"
 "P340"
 "P359"
 "P448"
 ⋮
 "P842"
 "P51"
 "P227"
 "P472"
 "P607"
 "P716"
 "P325"
 "P767"
 "P188"
 "P118"
 "P580"
 "P46"

In [11]:
patients = unique(C2_allmuts_strict.PID)

564-element Vector{String7}:
 "P50"
 "P69"
 "P75"
 "P122"
 "P151"
 "P216"
 "P252"
 "P254"
 "P273"
 "P289"
 "P340"
 "P359"
 "P448"
 ⋮
 "P842"
 "P51"
 "P227"
 "P472"
 "P607"
 "P716"
 "P325"
 "P767"
 "P188"
 "P118"
 "P580"
 "P46"

In [12]:
pats_single = setdiff!(patients, unique(C2_strict_multi_6h.patient_ID))

316-element Vector{String7}:
 "P69"
 "P75"
 "P216"
 "P273"
 "P289"
 "P340"
 "P359"
 "P658"
 "P712"
 "P768"
 "P908"
 "P914"
 "P920"
 ⋮
 "P842"
 "P51"
 "P227"
 "P472"
 "P607"
 "P716"
 "P325"
 "P767"
 "P188"
 "P118"
 "P580"
 "P46"

In [67]:
C2_single_strict = C2_allmuts_strict[ [x in pats_single for x in C2_allmuts_strict[!, :PID]] ,:]

,PID,Gene,Alt,Param,Amount
,String7,String15,String31,String31,Float64
1,P69,PRDM1,PRDM1:N172fs*17,k1_s_Blimp1-differentiation,0.5
2,P75,PRDM1,PRDM1:R442fs*64,k1_s_Blimp1-differentiation,0.5
3,P216,PRDM1,PRDM1:S76*,k1_s_Blimp1-differentiation,0.5
4,P273,PRDM1,PRDM1:Y248fs*3,k1_s_Blimp1-differentiation,0.5
5,P289,PRDM1,PRDM1:E153*,k1_s_Blimp1-differentiation,0.5
6,P340,PRDM1,PRDM1:K707fs*18,k1_s_Blimp1-differentiation,0.5
7,P359,PRDM1,PRDM1:R476fs*36,k1_s_Blimp1-differentiation,0.5
8,P658,PRDM1,PRDM1:S494fs*38,k1_s_Blimp1-differentiation,0.5
9,P712,PRDM1,PRDM1:L425fs*84,k1_s_Blimp1-differentiation,0.5


In [68]:
C2_single_strict[!, "ParamAmount"] = string.(C2_single_strict[!, "Param"], "_", C2_single_strict[!, "Amount"])

316-element Vector{String}:
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 ⋮
 "k2_5-cellCycle_0.5"
 "k1_s_IRF4-linking_1.5"
 "k1_mycExp-linking_1.5"
 "k1_mycExp-linking_1.5"
 "k1_mycExp-linking_1.5"
 "k2_scalingNumber-NFkB_1.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_Blimp1-differentiation_0.5"
 "k1_19-cellCycle_0.5"
 "k2_scalingNumber-NFkB_1.5"
 "k2_scalingNumber-NFkB_1.5"
 "k1_bcl2Exp-linking_0.5"

In [69]:
C2_single_strict

,PID,Gene,Alt,Param,Amount,ParamAmount
,String7,String15,String31,String31,Float64,String
1,P69,PRDM1,PRDM1:N172fs*17,k1_s_Blimp1-differentiation,0.5,k1_s_Blimp1-differentiation_0.5
2,P75,PRDM1,PRDM1:R442fs*64,k1_s_Blimp1-differentiation,0.5,k1_s_Blimp1-differentiation_0.5
3,P216,PRDM1,PRDM1:S76*,k1_s_Blimp1-differentiation,0.5,k1_s_Blimp1-differentiation_0.5
4,P273,PRDM1,PRDM1:Y248fs*3,k1_s_Blimp1-differentiation,0.5,k1_s_Blimp1-differentiation_0.5
5,P289,PRDM1,PRDM1:E153*,k1_s_Blimp1-differentiation,0.5,k1_s_Blimp1-differentiation_0.5
6,P340,PRDM1,PRDM1:K707fs*18,k1_s_Blimp1-differentiation,0.5,k1_s_Blimp1-differentiation_0.5
7,P359,PRDM1,PRDM1:R476fs*36,k1_s_Blimp1-differentiation,0.5,k1_s_Blimp1-differentiation_0.5
8,P658,PRDM1,PRDM1:S494fs*38,k1_s_Blimp1-differentiation,0.5,k1_s_Blimp1-differentiation_0.5
9,P712,PRDM1,PRDM1:L425fs*84,k1_s_Blimp1-differentiation,0.5,k1_s_Blimp1-differentiation_0.5


In [60]:
files = unique(C2_single_strict.ParamAmount)

16-element Vector{String}:
 "k1_s_Blimp1-differentiation_0.5"
 "k1_mycTsl-linking_1.5"
 "nik_deg_mod-NFkB_0.5"
 "k2_scalingNumber-NFkB_1.5"
 "k1_bcl2Exp-linking_1.5"
 "k1_26-cellCycle_0.5"
 "kmax_IkBaSynth-NFkB_0.5"
 "k1_s_IRF4-linking_0.5"
 "k1_76-apoptosis_0.5"
 "k1_e_pax5-differentiation_0.5"
 "k1_s_Bcl6-differentiation_1.5"
 "k2_5-cellCycle_0.5"
 "k1_s_IRF4-linking_1.5"
 "k1_mycExp-linking_1.5"
 "k1_19-cellCycle_0.5"
 "k1_bcl2Exp-linking_0.5"

In [61]:
C2_single_6hrdf = createEmptyCellDF()

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [62]:
C2_single_6hrdf = getFounderCells(files, "SSDoutputs/allMutSims/indivParams", C2_single_6hrdf, "mut")

cell missmatch in patient: k1_s_Blimp1-differentiation_0.5
3
1
found true cell 1: cell_42.jld2
cell missmatch in patient: k1_mycTsl-linking_1.5
5
1
found true cell 1: cell_16.jld2
cell missmatch in patient: k2_scalingNumber-NFkB_1.5
35
1
found true cell 1: cell_48.jld2
cell missmatch in patient: k1_bcl2Exp-linking_1.5
91
1
found true cell 1: cell_10.jld2
cell missmatch in patient: k1_26-cellCycle_0.5
5
1
found true cell 1: cell_10.jld2
cell missmatch in patient: kmax_IkBaSynth-NFkB_0.5
21
1
found true cell 1: cell_5.jld2
cell missmatch in patient: k1_s_IRF4-linking_0.5
25
1
found true cell 1: cell_26.jld2
cell missmatch in patient: k1_e_pax5-differentiation_0.5
3
1
found true cell 1: cell_54.jld2
cell missmatch in patient: k1_s_Bcl6-differentiation_1.5
13
1
found true cell 1: cell_2.jld2
cell missmatch in patient: k2_5-cellCycle_0.5
53
1
found true cell 1: cell_61.jld2
cell missmatch in patient: k1_mycExp-linking_1.5
71
1
found true cell 1: cell_12.jld2
couldn't process patient: k1_bcl

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt
,String,Float64,Float64,Float64,Float64,Float64
1,k1_s_Blimp1-differentiation_0.5,6.80562,15.7503,5.55301,64.4727,0.100766
2,k1_mycTsl-linking_1.5,6.80525,15.7503,5.55123,64.4697,0.100763
3,nik_deg_mod-NFkB_0.5,7.44864,17.2661,6.29013,68.8278,0.109509
4,k2_scalingNumber-NFkB_1.5,8.41863,19.6416,7.62555,74.8067,0.120167
5,k1_bcl2Exp-linking_1.5,6.80517,15.7504,5.55126,88.5857,0.100763
6,k1_26-cellCycle_0.5,6.80526,15.7509,5.55144,64.4696,0.100763
7,kmax_IkBaSynth-NFkB_0.5,9.81503,22.2481,9.89348,84.8429,0.137406
8,k1_s_IRF4-linking_0.5,6.80524,15.7503,5.55128,64.4697,0.100763
9,k1_76-apoptosis_0.5,6.8059,15.7527,5.552,64.472,0.100766


In [63]:
C2_single_6hrdf = unique(C2_single_6hrdf)

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt
,String,Float64,Float64,Float64,Float64,Float64
1,k1_s_Blimp1-differentiation_0.5,6.80562,15.7503,5.55301,64.4727,0.100766
2,k1_mycTsl-linking_1.5,6.80525,15.7503,5.55123,64.4697,0.100763
3,nik_deg_mod-NFkB_0.5,7.44864,17.2661,6.29013,68.8278,0.109509
4,k2_scalingNumber-NFkB_1.5,8.41863,19.6416,7.62555,74.8067,0.120167
5,k1_bcl2Exp-linking_1.5,6.80517,15.7504,5.55126,88.5857,0.100763
6,k1_26-cellCycle_0.5,6.80526,15.7509,5.55144,64.4696,0.100763
7,kmax_IkBaSynth-NFkB_0.5,9.81503,22.2481,9.89348,84.8429,0.137406
8,k1_s_IRF4-linking_0.5,6.80524,15.7503,5.55128,64.4697,0.100763
9,k1_76-apoptosis_0.5,6.8059,15.7527,5.552,64.472,0.100766


In [70]:
C2_strict_single_6h = createEmptyCellDF()

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [71]:
### assign 6h data to patients with single mutation
    npats_cumulative = 0
for i in 1:nrow(C2_single_6hrdf)
    param_df = DataFrame(C2_single_6hrdf[i, :])
    patients_df = C2_single_strict[(C2_single_strict.ParamAmount .== param_df.patient_ID),:]
    npats = nrow(patients_df)
    repeat!(param_df, npats)
    param_df[:, 1] = patients_df.PID
    append!(C2_strict_single_6h, param_df)
    npats_cumulative = npats_cumulative + npats
    println(npats_cumulative)
end

15
16
52
146
200
225
243
258
259
261
262
310
311
314
315


In [44]:
C2_strict_single_6h

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,P69,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
2,P75,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
3,P216,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
4,P273,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
5,P289,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
6,P340,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
7,P359,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
8,P658,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
9,P712,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884


In [45]:
C2_strict_6h = append!(C2_strict_multi_6h, C2_strict_single_6h)

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,P104,6.80557,15.7518,5.55172,88.5879,0.100765,0.0386653,1.46904
2,P105,10.4545,24.1281,10.4518,88.2749,0.144238,0.0441876,1.85993
3,P109,10.4528,24.1272,10.4453,88.2629,0.144226,0.0440745,1.86013
4,P111,6.80587,15.751,5.55325,64.4738,0.100768,0.0386937,1.46889
5,P112,10.4532,24.1287,10.4459,88.2643,0.144227,0.0440746,1.86023
6,P115,10.4532,24.1287,10.4459,88.2643,0.144227,0.0440746,1.86023
7,P117,6.80431,15.7506,5.54724,64.4627,0.100756,0.0385746,1.46915
8,P120,9.81549,22.2493,9.89401,84.8452,0.137409,0.0447539,2.04555
9,P122,8.41931,19.6415,7.62849,74.8121,0.120173,0.0413565,1.63945


In [46]:
sort!(C2_strict_6h, :patient_ID)

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,P1,6.8052,15.7507,5.55138,64.4694,0.100763,0.0386653,1.46895
2,P100,7.44864,17.2661,6.29013,68.8278,0.109509,0.0403151,2.14316
3,P102,8.41863,19.6416,7.62555,74.8067,0.120167,0.0413075,1.63959
4,P103,6.80517,15.7504,5.55126,88.5857,0.100763,0.0386653,1.46893
5,P104,6.80557,15.7518,5.55172,88.5879,0.100765,0.0386653,1.46904
6,P104,6.80557,15.7518,5.55172,88.5879,0.100765,0.0386653,1.46904
7,P105,10.4545,24.1281,10.4518,88.2749,0.144238,0.0441876,1.85993
8,P105,10.4545,24.1281,10.4518,88.2749,0.144238,0.0441876,1.85993
9,P108,8.41863,19.6416,7.62555,74.8067,0.120167,0.0413075,1.63959


In [72]:
setdiff!(C2_single_strict.PID, C2_strict_single_6h.patient_ID)

1-element Vector{String7}:
 "P46"

In [51]:
C2_single_strict[(C2_single_strict.PID .== "P46"),:]

,PID,Gene,Alt,Param,Amount,ParamAmount
,String7,String15,String31,String31,Float64,String
1,P46,PRDM1,PRDM1:N172fs*17,k1_s_Blimp1-differentiation,0.5,k1_s_Blimp1-differentiation_0.5


In [77]:
P46 = DataFrame(C2_single_6hrdf[(C2_single_6hrdf.patient_ID .== "k1_s_Blimp1-differentiation_0.5"),:])

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt
,String,Float64,Float64,Float64,Float64,Float64
1,k1_s_Blimp1-differentiation_0.5,6.80562,15.7503,5.55301,64.4727,0.100766


In [79]:
P46.patient_ID .= "P46"

1-element Vector{String}:
 "P46"

In [80]:
P46

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,P46,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884


In [81]:
append!(C2_strict_single_6h, P46)

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,P69,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
2,P75,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
3,P216,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
4,P273,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
5,P289,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
6,P340,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
7,P359,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
8,P658,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
9,P712,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884


In [82]:
unique(C2_strict_single_6h.patient_ID)

316-element Vector{String}:
 "P69"
 "P75"
 "P216"
 "P273"
 "P289"
 "P340"
 "P359"
 "P658"
 "P712"
 "P768"
 "P908"
 "P914"
 "P920"
 ⋮
 "P798"
 "P824"
 "P830"
 "P841"
 "P869"
 "P842"
 "P51"
 "P227"
 "P472"
 "P607"
 "P188"
 "P46"

In [83]:
C2_strict_all_6h = append!(C2_strict_multi_6h, C2_strict_single_6h)

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,P104,6.80557,15.7518,5.55172,88.5879,0.100765,0.0386653,1.46904
2,P105,10.4545,24.1281,10.4518,88.2749,0.144238,0.0441876,1.85993
3,P109,10.4528,24.1272,10.4453,88.2629,0.144226,0.0440745,1.86013
4,P111,6.80587,15.751,5.55325,64.4738,0.100768,0.0386937,1.46889
5,P112,10.4532,24.1287,10.4459,88.2643,0.144227,0.0440746,1.86023
6,P115,10.4532,24.1287,10.4459,88.2643,0.144227,0.0440746,1.86023
7,P117,6.80431,15.7506,5.54724,64.4627,0.100756,0.0385746,1.46915
8,P120,9.81549,22.2493,9.89401,84.8452,0.137409,0.0447539,2.04555
9,P122,8.41931,19.6415,7.62849,74.8121,0.120173,0.0413565,1.63945


In [84]:
sort!(C2_strict_all_6h, :patient_ID)

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,P1,6.8052,15.7507,5.55138,64.4694,0.100763,0.0386653,1.46895
2,P100,7.44864,17.2661,6.29013,68.8278,0.109509,0.0403151,2.14316
3,P102,8.41863,19.6416,7.62555,74.8067,0.120167,0.0413075,1.63959
4,P103,6.80517,15.7504,5.55126,88.5857,0.100763,0.0386653,1.46893
5,P104,6.80557,15.7518,5.55172,88.5879,0.100765,0.0386653,1.46904
6,P105,10.4545,24.1281,10.4518,88.2749,0.144238,0.0441876,1.85993
7,P108,8.41863,19.6416,7.62555,74.8067,0.120167,0.0413075,1.63959
8,P109,10.4528,24.1272,10.4453,88.2629,0.144226,0.0440745,1.86013
9,P110,6.8052,15.7507,5.55138,64.4694,0.100763,0.0386653,1.46895


In [85]:
CSV.write("C2_strict_all_6h.csv", C2_strict_all_6h)

"C2_strict_all_6h.csv"

In [86]:
C2_patient_info = DataFrame(CSV.File("C2_patient_info.csv"))

,PID,age_gt_60,IPI,ICDO3,rchop_treated,de_novo,cluster_AIC
,String7,String3,String31,String31,String3,String3,String15
1,P1,No,Low,Diffuse NOS,Yes,Yes,NOTCH2
2,P2,Yes,Missing,Diffuse NOS,No,Yes,NEC
3,P3,Yes,Missing,Diffuse NOS,No,No,BCL2
4,P4,Yes,Missing,Plasmablastic,No,Yes,NEC
5,P5,Yes,Intermediate/High,Diffuse NOS,Yes,Yes,NOTCH2
6,P6,Yes,Low,Diffuse NOS,Yes,Yes,SOCS1/SGK1
7,P7,Yes,Low,Diffuse NOS,Yes,Yes,NEC
8,P8,Yes,Missing,Diffuse NOS,Yes,No,NEC
9,P9,Yes,Low/Intermediate,Diffuse NOS,No,Yes,MYD88


In [87]:
C2_strict_pat_info = C2_patient_info[ [x in C2_strict_all_6h.patient_ID for x in C2_patient_info[!, :PID]] ,:]

,PID,age_gt_60,IPI,ICDO3,rchop_treated,de_novo,cluster_AIC
,String7,String3,String31,String31,String3,String3,String15
1,P1,No,Low,Diffuse NOS,Yes,Yes,NOTCH2
2,P2,Yes,Missing,Diffuse NOS,No,Yes,NEC
3,P5,Yes,Intermediate/High,Diffuse NOS,Yes,Yes,NOTCH2
4,P7,Yes,Low,Diffuse NOS,Yes,Yes,NEC
5,P8,Yes,Missing,Diffuse NOS,Yes,No,NEC
6,P9,Yes,Low/Intermediate,Diffuse NOS,No,Yes,MYD88
7,P12,Yes,High,Diffuse NOS,Yes,Yes,BCL2
8,P13,Yes,High,Diffuse NOS,Yes,Yes,NEC
9,P14,No,Missing,Diffuse NOS,Yes,Yes,SOCS1/SGK1


In [89]:
rename!(C2_strict_all_6h, :patient_ID => :PID)

,PID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,P1,6.8052,15.7507,5.55138,64.4694,0.100763,0.0386653,1.46895
2,P100,7.44864,17.2661,6.29013,68.8278,0.109509,0.0403151,2.14316
3,P102,8.41863,19.6416,7.62555,74.8067,0.120167,0.0413075,1.63959
4,P103,6.80517,15.7504,5.55126,88.5857,0.100763,0.0386653,1.46893
5,P104,6.80557,15.7518,5.55172,88.5879,0.100765,0.0386653,1.46904
6,P105,10.4545,24.1281,10.4518,88.2749,0.144238,0.0441876,1.85993
7,P108,8.41863,19.6416,7.62555,74.8067,0.120167,0.0413075,1.63959
8,P109,10.4528,24.1272,10.4453,88.2629,0.144226,0.0440745,1.86013
9,P110,6.8052,15.7507,5.55138,64.4694,0.100763,0.0386653,1.46895


In [91]:
C2_strict_all_6h_clin = innerjoin(C2_strict_pat_info, C2_strict_all_6h, on = :PID)

,PID,age_gt_60,IPI,ICDO3,rchop_treated
,String7,String3,String31,String31,String3
1,P1,No,Low,Diffuse NOS,Yes
2,P100,Yes,Low,Diffuse NOS,Yes
3,P102,Yes,Intermediate/High,Diffuse NOS,Yes
4,P103,No,Low,Diffuse NOS,Yes
5,P104,Yes,High,Diffuse NOS,Yes
6,P105,Yes,Missing,Diffuse NOS,Yes
7,P108,Yes,Missing,Diffuse NOS,No
8,P109,Yes,Missing,Diffuse NOS,Yes
9,P110,Yes,Low/Intermediate,Diffuse NOS,Yes


In [93]:
select!(C2_strict_all_6h_clin, Not([:ICDO3]))

,PID,age_gt_60,IPI,rchop_treated,de_novo,cluster_AIC,cluster_ICL
,String7,String3,String31,String3,String3,String15,String7
1,P1,No,Low,Yes,Yes,NOTCH2,NEC
2,P100,Yes,Low,Yes,Yes,BCL2,BCL2
3,P102,Yes,Intermediate/High,Yes,Yes,MYD88,MYD88
4,P103,No,Low,Yes,Yes,NOTCH2,NEC
5,P104,Yes,High,Yes,Yes,NOTCH2,SGK1
6,P105,Yes,Missing,Yes,Yes,MYD88,MYD88
7,P108,Yes,Missing,No,Yes,MYD88,MYD88
8,P109,Yes,Missing,Yes,Yes,NOTCH2,NEC
9,P110,Yes,Low/Intermediate,Yes,Yes,NEC,NEC


In [94]:
CSV.write("C2_strict_all_6h_clin.csv", C2_strict_all_6h_clin)

"C2_strict_all_6h_clin.csv"